# PCAP TO NPY

## Code #1

Complie Kitsune Libraries.

```
cd AfterImageExtractor/
python setup.py build_ext --inplace
```

Libraries
- https://github.com/dongtsi/TrafficManipulator
- https://github.com/ymirsky/Kitsune-py


In [1]:
from scapy.all import rdpcap
import numpy as np
from tabulate import tabulate
import pandas as pd
import sys
import os

In [2]:
input_path = "/mnt/Data/SPECTRE/Dataset/Malware_Capture_Facility_Project/2015-09-10_winlinux.pcap"
output_path = "/home/aryn/spectre-dev/dataset/Malware_Capture_Facility_Project/PCAP-NPY/2015-09-10_winlinux.npy" 

In [4]:
# TODO: Check module import from external directory.
import AfterImageExtractor.FEKitsune as Fe
from AfterImageExtractor.KitsuneTools import RunFE

pcap_file = input_path
feat_file = output_path

scapyin = rdpcap(pcap_file)

FE = Fe.Kitsune(scapyin, np.Inf)
feature, _ = RunFE(FE)

print(np.asarray(feature).shape)
np.save(feat_file, feature)

Importing AfterImage Cython Library
Importing Scapy Library
(539689, 100)


In [5]:
malicious_data = np.load(output_path, allow_pickle=True)
df = pd.DataFrame(malicious_data)

# Select the first two rows using iloc
two_rows = df.iloc[:2]

print(tabulate(two_rows, headers='keys', tablefmt='psql'))

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+---------+------+-------------+---------+------+-------------+---------+------+-------------+------+------+-------------+------+------+-------------+-------------+---------+-------------+-------------+------------+---------+------+------+-------------+---------+-------------+-------------+------------+---------+------+------+------------+---------+------+------+---------+-------------+-------------+---------+-------------+-------------+---------+-------------+-------------+------+-------------+-------------+------+-------------+-------------+---------+------+-------------+---------+------+-------------+---------+------+-------------+------+------+-------------+------+------+-------------+-------------+---------+-------------+-------------+------------+---------+------+------+-------------+---------+-------------+-------------+------------+---------+------+------+------------+---------+-

In [6]:
trained_data = np.load('/home/aryn/spectre-dev/dataset/ISCX-IDS-2012/PCAP-NPY/destinationPayload_TestbedSatJun12Flows.xml.npy', allow_pickle=True)
df = pd.DataFrame(trained_data)

# Select the first two rows using iloc
two_rows = df.iloc[:2]

print(tabulate(two_rows, headers='keys', tablefmt='psql'))

+----+-------------------------------+-----+
|    | 0                             |   1 |
|----+-------------------------------+-----|
|  0 | [ 42  32  79 ... 101 114 101] |   0 |
|  1 | [72 84 84 ... 58 53 57]       |   0 |
+----+-------------------------------+-----+


/home/aryn/miniconda3/envs/spectre/lib/python3.9/site-packages/tabulate/__init__.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (len(row) >= 1 and row[0] == SEPARATING_LINE)
/home/aryn/miniconda3/envs/spectre/lib/python3.9/site-packages/tabulate/__init__.py:108: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  or (len(row) >= 2 and row[1] == SEPARATING_LINE)


## Code #2

In [1]:
import numpy as np
import time
from scapy.all import *

In [6]:
errors = []

start_time = time.time()
data_array = np.empty((0, 2))
counter = 0
actual = (50**2) * 3
files = ["/mnt/Data/SPECTRE/Dataset/Malware_Capture_Facility_Project/2015-09-10_winlinux.pcap"]  # Replace with your list of PCAP files

for file in files:
    print(file)
    try:
        pcap = rdpcap(file)
        print('Reading File ', file)
    except:
        errors += file
        continue

    for packet in pcap:
        # Extract payload size
        payload_size = len(packet)

        # Process the payload size (similar to the code provided in the original question)
        if payload_size > actual:
            payload_size = actual
        else:
            while payload_size < actual:
                payload_size += payload_size
            payload_size = actual

        # Assign a label based on a specific condition (modify this as needed)
        if packet.haslayer(TCP):  # Example condition: if the packet has a TCP layer
            label = 0
        else:
            label = 1

        # Stack the processed payload size and label in the data_array
        data_array = np.vstack((data_array, np.array([[payload_size, label]])))

        counter += 1

    print('Time taken: {}'.format(time.time() - start_time))
    start_time = time.time()

    # Save the data_array as two separate files
    export_dataset = "/home/aryn/spectre-dev/dataset/Malware_Capture_Facility_Project/PCAP-NPY/2015-09-10_winlinux_2.npy"  # Replace with your export directory path
    np.save(export_dataset, np.array(data_array))

    data_array = np.empty((0, 2))

/mnt/Data/SPECTRE/Dataset/Malware_Capture_Facility_Project/2015-09-10_winlinux.pcap
Reading File  /mnt/Data/SPECTRE/Dataset/Malware_Capture_Facility_Project/2015-09-10_winlinux.pcap
Time taken: 347.2896406650543


In [9]:
from tabulate import tabulate

data_sample = np.load('/home/aryn/spectre-dev/dataset/Malware_Capture_Facility_Project/PCAP-NPY/2015-09-10_winlinux_2.npy', allow_pickle=True)

rows = data_sample[:2]
table = rows.tolist()
print(tabulate(table))

----  -
7500  1
7500  1
----  -
